In [5]:
import sqlite3
import pandas

<h3>pip install pyArango</h3>

In [6]:
from pyArango.connection import*
from pyArango.query import*

<h3>讀取sqlite3資料庫</h3>

In [1]:
data_path = '/Users/justin/Desktop/資料庫管理/data/W06_data'
db_file='%s/StockInfo.sqlite3' % data_path

In [2]:
file_names=[
    'StockID','StockDate','StockOpen',
    'StockHigh','StockLow','StockClose',
    'StockAdjClose','Stockvol'
]

In [12]:
sql="SELECT %s FROM StockInfo ORDER BY StockID,StockDate "%','.join(file_names)
print(sql)

SELECT StockID,StockDate,StockOpen,StockHigh,StockLow,StockClose,StockAdjClose,Stockvol FROM StockInfo ORDER BY StockID,StockDate 


db_conn_source=sqlite3.connect(db_file)
data=pandas.read_sql_query("SELECT %s FROM StockInfo ORDER BY StockID,StockDate "%','.join(file_names),db_conn_source)
db_conn_source.close()

In [9]:
def LoadDate(sql):
    global db_file
    _db_conn_source=sqlite3.connect(db_file)
    _data=pandas.read_sql_query(sql,_db_conn_source)
    _db_conn_source.close
    return _data

In [10]:
data=LoadDate(sql)

DatabaseError: Execution failed on sql 'SELECT StockID,StockDate,StockOpen,StockHigh,StockLow,StockClose,StockAdjClose,Stockvol FROM StockInfo ORDER BY StockID,StockDate ': no such table: StockInfo

In [28]:
df_index=[]
for i in range(data.shape[0]):
    row=data.iloc[i]
    idx='%s-%s'%(row['StockDate'],row['StockID'])
    df_index.append(idx)
data.index=df_index

In [29]:
data.head()

,StockID,StockDate,StockOpen,StockHigh,StockLow,StockClose,StockAdjClose,StockVol
2018-01-02-IBM,IBM,2018-01-02,154.500000,154.809998,153.539993,154.250000,149.454422,4202500
2018-01-03-IBM,IBM,2018-01-03,157.339996,159.809998,156.330002,158.490005,153.562622,9441600
2018-01-04-IBM,IBM,2018-01-04,159.649994,162.320007,159.369995,161.699997,156.672821,7556200
2018-01-05-IBM,IBM,2018-01-05,162.440002,162.899994,161.100006,162.490005,157.438263,5195800
2018-01-08-IBM,IBM,2018-01-08,162.660004,163.910004,161.699997,163.470001,158.387787,5237500


<h3>ArangoDB 資料庫</h3>

In [11]:
db_ip='localhost'

db_conn=Connection(
    arangoURL='http://%s:8529'%db_ip,
    username='emprogria',
    password='emprogria'
)

ConnectionError: Unauthorized access, you must supply a (username, password) with the correct credentials. URL: http://localhost:8529/_api/user/emprogria/database, status: 401. Errors: b'{"error":true,"errorNum":11,"errorMessage":"not authorized to execute this request","code":401}'

In [13]:
db=db_conn['Emprogria']
db_collection=db.collections['StockInfo2']

NameError: name 'db_conn' is not defined

<h3>儲存ArangoDB資料庫</h3>

In [50]:
db_result=db_collection.empty()
data.iloc[0]

StockID                 IBM
StockDate        2018-01-02
StockOpen             154.5
StockHigh        154.809998
StockLow         153.539993
StockClose           154.25
StockAdjClose    149.454422
StockVol            4202500
Name: 2018-01-02-IBM, dtype: object

In [51]:
for i in range(data.shape[0]):
    row=data.iloc[i]
    
    _doc=db_collection.createDocument()
    _doc['_key']=data.index[i]
    
    for col_name in data.columns:
        _doc[col_name]=row[col_name]
    _doc.save()

In [52]:
db_conn.close()

AttributeError: 'Connection' object has no attribute 'close'